### 텐서플로우를 이용하여 ML 구현

- 알고리즘 SVM을 이용하여 BMI를 학습시키고, 비만도 판정
- BMI 지수 = (몸무게/키)*키
- 데이터 : 3_bmi.csv

<img src = './data/텐서용어.png'>

In [2]:
# 사용 패키지
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 데이터 로드
csv = pd.read_csv('./data/3_bmi.csv')
csv.head()

,height,weight,label
0,178,69,normal
1,190,62,thin
2,134,61,fat
3,144,43,normal
4,149,40,thin


In [4]:
csv.shape

(20000, 3)

In [5]:
# 데이터 전처리
# height	weight의 최대값 기준 정규화
csv['height'].max(), csv['weight'].max()

(200, 80)

In [6]:
csv['height'] = csv['height'] / csv['height'].max()
csv['weight'] = csv['weight'] / csv['weight'].max()

In [7]:
csv.head()

,height,weight,label
0,0.890,0.8625,normal
1,0.950,0.7750,thin
2,0.670,0.7625,fat
3,0.720,0.5375,normal
4,0.745,0.5000,thin


In [8]:
csv['label'].unique()

array(['normal', 'thin', 'fat'], dtype=object)

In [9]:
# csv['label']의 벡터화 → 분류변수
labelClass = {
    'thin' : [1, 0, 0],
    'normal' : [0, 1, 0],
    'fat' : [0, 0, 1]
}
labelClass

{'thin': [1, 0, 0], 'normal': [0, 1, 0], 'fat': [0, 0, 1]}

In [10]:
# 딕셔너리 값 추출
labelClass['thin']

[1, 0, 0]

In [16]:
# csv['label'].apply(lambda x : labelClass[x])[:5]
# print( csv['label'].apply(lambda x : labelClass[x])[:5] )
# csv['label'].apply(lambda x : np.array(labelClass[x]))[:5]
# print( csv['label'].apply(lambda x : np.array(labelClass[x]))[:5] )

In [17]:
# 데이터 전환
# → 값으로 배열을 넣는다 
# → np.array(리스트)
csv['label_bmi'] = csv['label'].apply(lambda x : np.array(labelClass[x])) 

In [18]:
csv.head()

,height,weight,label,label_bmi
0,0.890,0.8625,normal,"[0, 1, 0]"
1,0.950,0.7750,thin,"[1, 0, 0]"
2,0.670,0.7625,fat,"[0, 0, 1]"
3,0.720,0.5375,normal,"[0, 1, 0]"
4,0.745,0.5000,thin,"[1, 0, 0]"


In [19]:
# 데이터 준비 → 훈련용 데이터, 검증용 데이터로 분류
from sklearn.model_selection import train_test_split

In [21]:
# DataFrame : 차원유지
csv[ ['height', 'weight'] ][:5]

,height,weight
0,0.890,0.8625
1,0.950,0.7750
2,0.670,0.7625
3,0.720,0.5375
4,0.745,0.5000


In [22]:
# Series : 차원축소
csv['label_bmi'][:5]

0    [0, 1, 0]
1    [1, 0, 0]
2    [0, 0, 1]
3    [0, 1, 0]
4    [1, 0, 0]
Name: label_bmi, dtype: object

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    csv[ ['height', 'weight'] ],  # DataFrame : 차원유지
    csv['label_bmi'],  # Series : 차원축소
    test_size = 0.25,
    random_state = 0   # 일관성 있는 결과를 얻고 싶다면, 난수의 시드를 고정
)

In [38]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15000, 2), (5000, 2), (15000,), (5000,))

In [39]:
X_train[:2]

,height,weight
16152,0.91,0.8125
17768,0.75,0.8375


In [40]:
y_train[:2]

16152    [0, 1, 0]
17768    [0, 0, 1]
Name: label_bmi, dtype: object

- 텐서플로우 등장
- 소프트맥스 회귀라는 알고리즘을 사용 → DL 수행 시 자주 사용하는 활성화 함수 중에 하나
- softmax(), sigmod(), ... 
- 인공신경망과 아주 유사

<img src = './data/sm0.png' width = '200'>

- 각각의 입력값에 가중치(W)를 곱하고, bias(b)를 더하고 나서 sortmax() 함수를 통과시키면, 출력값이 나타나게 된다

<img src = './data/sm1.png' width = '500'>

<img src = './data/sm2.png' width = '500'>

<img src = './data/sm3.png' width = '500'>

- **y = softmax(Wx+b)**
- 실제 텐서플로우로 구현 시, 차원유지를 위해서 Wx는 xW로 구현할 수 있다 → 행렬곱

In [27]:
# 소프트맥스 회귀 식을 텐서플로우로 구현하기 위해 구성요소 정의
# 입력(x) → 키, 몸무게
# 입력 데이터 N개
x = tf.placeholder( tf.float32, [None, 2] )

In [28]:
# 출력 y_ → 1.0.0 or 0.1.0 or 0.0.1 → [None, 3]
y_ = tf.placeholder( tf.float32, [None, 3] )

In [29]:
# bias(편향)
# tf.zeros()를 통해서 해당 행렬에 기본값을 0으로 세팅
b = tf.Variable( tf.zeros([3]) )

Instructions for updating:
Colocations handled automatically by placer.


In [30]:
# 가중치 : W → 필터, 커널로 표현
# [None, 2]*W + b = [None, 3]
# [None, 2]*[2, 3] + [3] = [None, 3]
# tf.zeros()를 통해서 해당 행렬에 기본값을 0으로 세팅
W = tf.Variable( tf.zeros([2, 3]) )

In [31]:
# 데이터 플로우 그래프 y
y = tf.nn.softmax( tf.matmul( x, W ) + b )

In [41]:
'%s x %s + %s = %s' % (x.shape, W.shape, b.shape, y.shape)

'(?, 2) x (2, 3) + (3,) = (?, 3)'

#### 학습

- 좋은 모델에 대한 정의
- 기준
    - 비용(cost), 손실(loss) 등 원하는 결과에서 얼마나 떨어져 있는지 보여주는 지표
    - 이런 격차를 줄이는 방향으로 내용 전개
    - 이를 위해서 통상 '크로스 엔트로피' 알고리즘을 적용
    - 정보 이론 분야에 정보 압축 알고리즘으로 고안
    - 도박, 머신러닝 등에 중요한 아이디어로 사용
    <img src = './data/sm4.png' width='200'>

In [42]:
# 텐서플로우로 크로스 엔트로피 구성
# y_ : 정답 레이블, y : 예측 레이블
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

In [43]:
# 크로스 엔트로피 값을 최소화하도록 작업이 진행
# → 경사 하강법(gradient descent algorithm)으로 수행
# 경사하강법을 적용하면 
# 텐서플로우가 각각의 변수를 비용을 줄이는 방향으로 조금씩 이동시켜서 연산
# → 오차율이 최소가 되도록

# 경사 하강법 알고리즘 생성
# 하이퍼파라미터 0.01
optimizer = tf.train.GradientDescentOptimizer( 0.01 )

In [44]:
# 훈련 정의
train = optimizer.minimize( cross_entropy )
train

<tf.Operation 'GradientDescent_1' type=NoOp>

In [49]:
# tf.argmax( y, 1 ), tf.argmax(y_, 1)

In [36]:
# 정답률 예측 정의
# y_ : 정답 레이블
# y : 예측 레이블
# tf.argmax( y, 1 ) : 모델이 판단하기에 각 데이터별로 가장 적합하다고 판단되는 라벨
predict = tf.equal(tf.argmax( y, 1 ), tf.argmax(y_, 1))
predict

<tf.Tensor 'Equal:0' shape=(?,) dtype=bool>

In [50]:
# 정확도 정의
# predict(불린형 리스트) → 부동소수로 변환 → 평균
accuracy = tf.reduce_mean( tf.cast(predict, tf.float32) )

- 세션 가동 → 실제 연산 수행 → 결과 획득

In [51]:
X_train[ 0 : 100 ][['height', 'weight']].head(2)

,height,weight
16152,0.91,0.8125
17768,0.75,0.8375


In [81]:
# y_train[0 : 100]

In [82]:
# list(y_train[0 : 100])

In [80]:
TERM = 100
size = int(150/TERM)
# print(size)

startIdxs = []
for step in range(size * 10) :
    startIdx = int(step * TERM / 10)
    startIdxs.append(startIdx)
    print(startIdx, startIdx+TERM)

print(len(startIdxs))

0 100
10 110
20 120
30 130
40 140
50 150
60 160
70 170
80 180
90 190
10


In [83]:
# 구동을 반복해서 시키면 학습량이 증가해서
# 정확도가 상승한다
with tf.Session() as sess :
    
    # 변수 초기화
    sess.run( tf.global_variables_initializer() )
    print( 'W', W )
    print( 'b', b )
    
    TERM = 100
    size = int(X_train.shape[0]/TERM)  
    print('b', size)
    
    # 학습
    # 성능 향상(일반적)
    for step in range( size * 16 ) :
        # X_train에서 데이터를 추출 → 시작 위치
        startIdx = int(step * TERM / 16)
        # 앞에서부터 TERM 단위로 훈련 데이터를 추출하겠다
        rows = X_train[ startIdx : startIdx + TERM ]
        
        # 학습 데이터 구성
        x_pat = rows[ ['height', 'weight'] ]
        y_ans = list(y_train[ startIdx : startIdx + TERM ])
        
        # 평가를 위한 구조
        fd = { x : x_pat, y_ : y_ans }
        
        # 학습 세트 훈련
        # sess.run 구동 시 데이터를 전달 → feed_dict
        sess.run( train, feed_dict=fd )
        
        # 중간 확인
        if step % 15 == 0 :
            # 크로스 엔트로피값 획득
            cre = sess.run(cross_entropy, feed_dict = fd)
            # 정확도
            acc = sess.run(accuracy, feed_dict = {x:X_test, y_: list(y_test)})
            print('step = %s, cre = %s, acc = %s' %(step, cre, acc))
    
    # 최종적인 정답값 확인
    acc = sess.run(accuracy, feed_dict = {x:X_test, y_:list(y_test)})
    print('정답률', acc)
    
    # 텐서들의 관계 및 흐름을 보기 위해 지원 : 텐서보드
    tf.summary.FileWriter('./bmi_tf_log', graph=sess.graph)

W <tf.Variable 'Variable_1:0' shape=(2, 3) dtype=float32_ref>
b <tf.Variable 'Variable:0' shape=(3,) dtype=float32_ref>
b 150
step = 0, cre = 106.28908, acc = 0.3788
step = 15, cre = 98.79806, acc = 0.3968
step = 30, cre = 94.22411, acc = 0.6388
step = 45, cre = 91.32673, acc = 0.7412
step = 60, cre = 89.753494, acc = 0.793
step = 75, cre = 84.77294, acc = 0.5854
step = 90, cre = 80.034706, acc = 0.6806
step = 105, cre = 78.591446, acc = 0.6886
step = 120, cre = 75.05971, acc = 0.8148
step = 135, cre = 75.22493, acc = 0.7664
step = 150, cre = 75.71864, acc = 0.8212
step = 165, cre = 60.266014, acc = 0.6856
step = 180, cre = 68.56894, acc = 0.6824
step = 195, cre = 65.36645, acc = 0.6894
step = 210, cre = 67.419556, acc = 0.7324
step = 225, cre = 63.54231, acc = 0.7344
step = 240, cre = 65.125626, acc = 0.8458
step = 255, cre = 68.07184, acc = 0.8666
step = 270, cre = 60.797073, acc = 0.715
step = 285, cre = 59.59044, acc = 0.7498
step = 300, cre = 56.079666, acc = 0.704
step = 315, cre

In [ ]:
# 텐서보드 가동
# shell을 가동
# !tensorboard --logdir=bmi_tf_log

# 컴퓨터에 따라서 실행이 안 될 수 있으니
$ tensorboard --logdir = bmi_tf_log